In [1]:
import pandas as pd

columns = [
        'join_id','dt','chain'
        ,'num_l1_txs_inbox','num_l1_txs_output','l1_gas_used_inbox','l1_gas_used_output','l1_gas_used_combined','calldata_bytes_l1_inbox'
        ,'l2_num_txs_per_day','l1_gas_used_on_l2','calldata_bytes_l2_per_day'
        ]

In [2]:
#Unify datasets
dunedf = pd.read_csv('csv_inputs/op_chain_gas_economics_dune_query_2453515.csv')
goldskydf = pd.read_csv('csv_inputs/op_chain_gas_economics_goldsky_zora_pgn.csv')

In [3]:
#Configure
#rename cols
dunedf = dunedf.rename(columns={
                        'name':'chain'
                        })
#Generate Join ID Column
dunedf['join_id'] = dunedf['dt'].astype(str).str[:10].str.cat(dunedf['chain'].astype(str).str.lower())
goldskydf['join_id'] = goldskydf['dt'].astype(str).str[:10].str.cat(goldskydf['chain'].astype(str).str.lower())

# display(dunedf.head(10))
# print('---')
# display(goldskydf.head(10))

combo_df = dunedf.merge(goldskydf, on='join_id', how='left')

# display(combo_df)

for c in columns:
        try:
                combo_df[c] = combo_df[c+'_x'].combine_first(combo_df[c+'_y']) #pick first non-null
        except:
                combo_df[c] = combo_df[c] #nada

combo_df['dt'] = pd.to_datetime(combo_df['dt'])
combo_df = combo_df[columns]

In [4]:
display(combo_df)

combo_df.to_csv('outputs/op_chain_gas_economics_sample.csv')

,join_id,dt,chain,num_l1_txs_inbox,num_l1_txs_output,l1_gas_used_inbox,l1_gas_used_output,l1_gas_used_combined,calldata_bytes_l1_inbox,l2_num_txs_per_day,l1_gas_used_on_l2,calldata_bytes_l2_per_day
0,2023-08-25base,2023-08-25 00:00:00+00:00,Base,1440,24,1.444513e+09,2084328.0,1.446597e+09,88661392.0,667309.0,2.349967e+09,139232837.0
1,2023-08-25op mainnet,2023-08-25 00:00:00+00:00,OP Mainnet,573,24,1.068661e+09,2084616.0,1.070745e+09,66223249.0,328347.0,1.642750e+09,129204211.0
2,2023-08-25zora network,2023-08-25 00:00:00+00:00,Zora Network,146,24,1.361066e+08,2084316.0,1.381910e+08,8340159.0,56677.0,1.885466e+08,22535690.0
3,2023-08-25public goods network,2023-08-25 00:00:00+00:00,Public Goods Network,146,24,3.777366e+07,2084316.0,3.985798e+07,2173015.0,850.0,9.909352e+06,12395413.0
4,2023-08-24base,2023-08-24 00:00:00+00:00,Base,1448,24,1.634661e+09,2084340.0,1.636746e+09,100567568.0,682907.0,2.585008e+09,162336376.0
...,...,...,...,...,...,...,...,...,...,...,...,...
268,2023-06-10op mainnet,2023-06-10 00:00:00+00:00,OP Mainnet,761,24,1.337027e+09,2084616.0,1.339111e+09,82801904.0,428588.0,2.181673e+09,174293806.0
269,2023-06-09op mainnet,2023-06-09 00:00:00+00:00,OP Mainnet,725,24,1.266183e+09,2084616.0,1.268268e+09,78404424.0,421777.0,2.053902e+09,164068467.0
270,2023-06-08op mainnet,2023-06-08 00:00:00+00:00,OP Mainnet,680,24,1.175501e+09,2084604.0,1.177585e+09,72779082.0,394920.0,1.916154e+09,153627516.0
271,2023-06-07op mainnet,2023-06-07 00:00:00+00:00,OP Mainnet,665,24,1.138901e+09,2084532.0,1.140985e+09,70508146.0,381914.0,1.826045e+09,139894360.0
